Fonctionnement du script
------------
Le script permet de créer dans un fichier FASTA toutes les mutations qui s'appliquent pour chaque transcrit pour chaque gène contenu dans la base de données de COSMICS, dans le but d'obtenir une prédiction du CDS avant et après la mutation à l'aide de FOMONET.

Chaque cellule doit, à tour de rôle et dans l'ordre, être roulée.

À quelques endroits dans le script, on retrouve des prints qui indique si tout fonctionne bien ou s'il y a des erreurs.

À la fin, c'est le fichier '../result/all_transcripts_with_mutation.fasta' qui doit être rouler dans FOMOnet. Il contient tous les transcrits de chaque gène à l'état WILD-TYPE en plus de tous les transcrits des gènes contenu dans COSMICS avec chacune des mutations. (Il y a toujours une seule mutation effectuée dans chaque transcrit (PAS DE COMBINAISON DE MUTATION))


In [ ]:
# RELIER LE NOM USUEL DU GÈNE (EX. BRCA1) UTILISÉ SUR COSMICS À SON TRANSCRIT DE RÉFÉRENCE
les_transcrits_de_reference = {}
import re
data = '../data/Cosmic_MutantCensus_v101_GRCh38.tsv' # Catalogue COSMICS: contient toutes les mutations de tous les gènes
with open(data, 'r') as file: 
    next(file) # Skip la première ligne (header)
    for line in file:
        colonne = line.split('\t')
        gene_name = colonne[0] # Nom usuel sur COSMICS (ex. BRCA1)
        transcrit_name_info = re.search(r'ENST[0123456789]{5,100}', colonne[2])
        transcrit_reference = transcrit_name_info.group(0) # Transcrit de référence, les mutations sont établies selon ce transcrit
        les_transcrits_de_reference[gene_name]= transcrit_reference # Clé = nom usuel du gène, Valeur = son transcrit de référence

# RELIER LE NOM USUEL DU GÈNE (EX. BRCA1) UTILISÉ SUR COSMICS À SON IDENTIFIANT ENSG     
equivalence_nom = {}
with open('../data/Homo_sapiens.GRCh38.113.gtf', 'r') as file: # GTF
    for line in file:
        colonne_tab = line.split('\t')
        colonne_guillemet = line.split('"')
        if len(colonne_tab) > 3: # Pour enlever les trois premières lignes du file qui n'ont pas rapport
            for valeur in les_transcrits_de_reference.values(): # Itération sur le transcrit de référence pour tous les gènes
                if colonne_tab[2] == 'transcript' and valeur in line: # Sélectionne la ligne du transcrit de référence
                    gene = colonne_guillemet[1] # Identifiant ENSG du gène
                    list_gene_name = [cle for cle, v in les_transcrits_de_reference.items() if v == valeur] # Je recherche le nom usuel du gène pour lequel je viens de trouver le transcrit de référence
                    cosmic_gene_name = list_gene_name[0]
                    equivalence_nom[cosmic_gene_name]= gene # Établit un lien entre le nom usuel et ENSG
#print(equivalence_nom)

# Il y a 35 gènes/741 gènes présents dans la database de Cosmic dont le transcrit de référence ne se retrouve pas dans le GTF. 
# COSMICS utilise uniquement le nom usuel du gène dans sa database et le GTF utilise l'identifiant ENSG, pour l'instant, mon code établit le lien
# nom usuel-identifiant ENSG grâce à l'identifiant ENST du transcrit de référence.
# Ces gènes ne pourront pas être effectué pour le moment, car Cosmics donne ses positions de mutations par rapport au transcrit de référence.
# Il reste 706 gènes qui sont traités. 

{'PRDM16': 'ENSG00000142611', 'THRAP3': 'ENSG00000054118', 'CDKN2C': 'ENSG00000123080', 'TENT5C': 'ENSG00000183508', 'TPR': 'ENSG00000047410', 'PRRX1': 'ENSG00000116132', 'ARHGEF10L': 'ENSG00000074964', 'ID3': 'ENSG00000117318', 'ATP1A1': 'ENSG00000163399', 'H3F3A': 'ENSG00000163041', 'ARNT': 'ENSG00000143437', 'MPL': 'ENSG00000117400', 'ASPM': 'ENSG00000066279', 'AKT3': 'ENSG00000117020', 'PRCC': 'ENSG00000143294', 'TNFRSF14': 'ENSG00000157873', 'RBM15': 'ENSG00000162775', 'STIL': 'ENSG00000123473', 'NRAS': 'ENSG00000213281', 'MLLT11': 'ENSG00000213190', 'FH': 'ENSG00000091483', 'BCL9': 'ENSG00000116128', 'ELF3': 'ENSG00000163435', 'RGS7': 'ENSG00000182901', 'MDM4': 'ENSG00000198625', 'NOTCH2': 'ENSG00000134250', 'PIK3R3': 'ENSG00000117461', 'SETDB1': 'ENSG00000143379', 'PRDM2': 'ENSG00000116731', 'TAL1': 'ENSG00000162367', 'FCGR2B': 'ENSG00000072694', 'SFPQ': 'ENSG00000116560', 'FCRL4': 'ENSG00000163518', 'DDR2': 'ENSG00000162733', 'LCK': 'ENSG00000182866', 'S100A7': 'ENSG00000143556

In [ ]:
# FAIRE LE DICTIONNAIRE GENE_DIC/GENE_DIC2

# Ce dictionnaire contient tous les gènes présents dans le catalogue cosmics. Tous les transcrits du gènes présents dans le GTF et tous les exons
# faisant partie de chaque transcrit, ainsi que leur position de départ et de fin sont ajoutés dans le dictionnaire.

#count = 0 # Sert à arrêter le code (Ne pas générer les 706 gènes)
import re
nature_du_brin = {} 
gene_dic = {}
data = '../data/Homo_sapiens.GRCh38.113.gtf' # GTF
with open(data, 'r') as file:
    for line in file:
        # Séparation dans les lignes à l'aide des caractères '\t' et '"'
        colonne_tab = line.split('\t') 
        colonne_guillemet = line.split('"')
            
        if len(colonne_tab) >= 3 and len(colonne_guillemet) >= 3: # Retire les trois premières lignes du file qui n'ont pas rapport

            if colonne_tab[2]== 'gene': # Seulement les lignes des gènes
                gene = colonne_guillemet[1] # Identifiant du gène ENSG
                
                for valeur in equivalence_nom.values(): # Itération sur les identifiants ENSG qui font parties du catalogue COSMIC
                    if valeur == gene:
                        count += 1
                        brand = colonne_tab[6] # Nature du brin du gène
                        nature_du_brin[gene]= brand # Dictionnaire qui relie le nom du gène ENSG à la nature du brin sur lequel il se retrouve
                        gene_dic[gene]= {} # Ajoute le nom du gène ENSG comme clé dans le gene_dic
            
            # Itération sur chaque gène ajouté dans le gene_dic (ça veut dire qu'il fait partie du catalogue)     
            for gene in gene_dic:
                if gene in line: 
                    if colonne_tab[2]== 'transcript': 
                        transcrit_nom = colonne_guillemet[5] # Nom du transcrit ENST
                        gene = colonne_guillemet[1] # Nom du gène ENSG
                        gene_dic[gene][transcrit_nom]= {} # Ajoute les transcrits sous les bons gènes
                    if colonne_tab[2]== 'exon':
                        gene = colonne_guillemet[1] # Nom du gène ENSG
                        transcrit_name_info = re.search(r'ENST[0123456789]{5,100}', line)
                        transcrit_name = transcrit_name_info.group(0) # Nom du transcrit ENST
                        exon_name_info = re.search(r'ENSE[0123456789]{5,100}', line)
                        exon_name = exon_name_info.group(0) # Nom de l'exon ENSE
                
                        # Positions de départ et de fin de l'exon
                        petit = colonne_tab[3] # Position génomique la plus petite
                        grand = colonne_tab[4] # Position génomique la plus grande

                        for key in gene_dic[gene].keys(): # Itère sur les transcrits faisant partie d'un gène
                            if key == transcrit_name: 
                                gene_dic[gene][transcrit_name][exon_name]= [] # Ajoute les exons dans les bons transcrits

                        for key in gene_dic[gene][transcrit_name].keys(): # Itère sur les exons faisant partie des transcrits
                            if key == exon_name: 
                                
                                # Ajoutes les positions de début et de fin de l'exon
                                gene_dic[gene][transcrit_name][exon_name].append(petit) 
                                gene_dic[gene][transcrit_name][exon_name].append(grand)

                                
# Il serait pratique d'avoir un gene_dic2 qui utilise le nom usuel du gène (ex. BRCA1) au lieu de son identifiant ENSG, car la base COSMICS n'utilise
# pas l'identifiant ENSG.

modif_equivalence_nom = {valeur: cle for cle, valeur in equivalence_nom.items()} # Cette ligne permet d'inverser les clés et les valeurs dans le dictionnaire equivalence_nom
nature_du_brin2 = {modif_equivalence_nom.get(cle, cle): valeur for cle, valeur in nature_du_brin.items()}
gene_dic2 = {modif_equivalence_nom.get(cle, cle): valeur for cle, valeur in gene_dic.items()}


##### À RETIRER LORSQUE LE SCRIPT SERA TERMINÉ
#gene_dic.popitem()
#gene_dic2.popitem()


#print(nature_du_brin)
#print(nature_du_brin2)
#print(gene_dic) # Avec identifiant ENSG des gènes
#print(gene_dic2) # Avec nom usuel COSMICS des gènes

{'ENSG00000142611': '+', 'ENSG00000054118': '+', 'ENSG00000123080': '+', 'ENSG00000183508': '+', 'ENSG00000047410': '-', 'ENSG00000116132': '+', 'ENSG00000074964': '+', 'ENSG00000117318': '-'}
{'PRDM16': '+', 'THRAP3': '+', 'CDKN2C': '+', 'TENT5C': '+', 'TPR': '-', 'PRRX1': '+', 'ARHGEF10L': '+', 'ID3': '-'}
{'ENSG00000142611': {'ENST00000511072': {'ENSE00002048533': ['3069168', '3069296'], 'ENSE00001754112': ['3186125', '3186474'], 'ENSE00003480863': ['3244087', '3244137'], 'ENSE00002034212': ['3385149', '3385286'], 'ENSE00003700221': ['3396491', '3396593'], 'ENSE00003696962': ['3402791', '3402998'], 'ENSE00003700688': ['3404739', '3404886'], 'ENSE00003700645': ['3405495', '3405648'], 'ENSE00003695658': ['3411384', '3412800'], 'ENSE00003701451': ['3414560', '3414647'], 'ENSE00003699052': ['3417828', '3417997'], 'ENSE00003698430': ['3418667', '3418744'], 'ENSE00003699796': ['3425581', '3425750'], 'ENSE00003701891': ['3426051', '3426225'], 'ENSE00003698226': ['3430872', '3431108'], 'ENS

In [9]:
# VÉRIFICATION DU GENE_DIC ET GENE_DIC2 GÉNÉRÉ
# Permet de vérifier que le nombre de gène généré dans le gene_dic est égale à celui dans la database de Cosmics

if len(equivalence_nom)== len(gene_dic2):
    print('Correct! Le nombre de gène à traiter est de ' + str(len(equivalence_nom)) + ' et le gene_dic contient ' + str(len(gene_dic)) + ' gènes')
else:
    print('Erreur! Le nombre de gène à traiter est de ' + str(len(equivalence_nom)) + ' et le gene_dic contient ' + str(len(gene_dic)) + ' gènes')
                

Erreur! Le nombre de gène à traiter est de 706 et le gene_dic contient 7 gènes


In [3]:
def sequence_cDNA (gene, transcrit):
    """
    Sortir la séquence d'ADNc d'un transcrit d'un gène.
    
    Paramètres
    ----------
        gene : nom du gène (ENSG)
        transcrit : nom du transcrit (ENST)
            
    Returns
    -------
        la séquence d'ADNc du transcrit d'un gène
    """
    header_of_transcrit = ''

    with open('../data/Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file:  # Toutes les séquences ADNc de chaque transcrit de tous les gènes
        for line in file:
            if gene in line and transcrit in line:
                header_of_transcrit = line.strip() # Stock le header de la séquence recherché
                break 

    if header_of_transcrit:
        sequence = "" 
        header_trouve = False

        with open('../data/Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file:
            for line in file:
                line = line.strip()
                if line.startswith(">"):
                    if header_trouve:
                        break

                    if line == header_of_transcrit:
                        header_trouve = True

                elif header_trouve:
                    sequence += line # Stock la séquence d'ADNc du transcrit

        if sequence: # Si la séquence a été trouvé
            return sequence
            
        else: # Si aucune séquence n'a été trouvé
            return ("Séquence non trouvée.")
            
    else: # Si aucun header n'a été trouvé
        return (f"Le header {header_of_transcrit} n'a pas été trouvé.")

In [4]:
def get_position_chronologique(gene, position_chromosomique, transcrit): 
    """
    Convertir une position génomique en position transcriptomique. N.B. première position = 1.
    
    Paramètres
    ----------
        gene : nom du gène (ENSG)
        position_chromosomique : position dans le génome
        transcrit : nom du transcrit (ENST)
            
    Returns
    -------
        la position transcriptomique
    """    
    total = 0
    liste_position_fin_exon = [0] # Liste qui va contenir la position de fin de chaque exon en coordonnées chronologiques

    for exon in gene_dic[gene][transcrit]:
        
        # Calcul de la longueur de l'exon en nombre de nucléotides
        brand = nature_du_brin[gene]
        longueur_exon = int(gene_dic[gene][transcrit][exon][1])-int(gene_dic[gene][transcrit][exon][0])+1 
        total = total + int(longueur_exon) # Cette variable finit par contenir la longueur de l'exon et tous les exons précédents = position chronologique de fin pour chaque exon
        liste_position_fin_exon.append(total) # Ajoute les positions chronologiques de fin pour chaque exon

        # CONDITIONS: la position génomique doit être dans un exon
        if position_chromosomique >= int(gene_dic[gene][transcrit][exon][0]) and position_chromosomique <= int(gene_dic[gene][transcrit][exon][1]): 
            keys = list(gene_dic[gene][transcrit].keys()) # Stock le nom de l'exon ENSE présentemment itéré
            position_keys = keys.index(exon) # Stock position de l'exon dans le transcrit

            # Pour un gène sur le brin négatif
            if brand == '-':
                # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(int(gene_dic[gene][transcrit][exon][1])-position_chromosomique) 

            # Pour un gène sur le brin positif
            if brand == '+':
                # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(position_chromosomique-int(gene_dic[gene][transcrit][exon][0]))
    return position_chronologique

In [5]:
# RÉPARTITION DES MUTATIONS SI ELLES TOMBENT DANS UN EXON, DANS UN CHEVAUCHEMENT INTRON-EXON OU DANS UN INTRON ET DE QUEL TYPE DE MUTATION IL S'AGIT ( SUBS, INV, INS, DEL, DELINS, DUP)

# Triage des mutations pour chaque gène

# Cette première section sert à supprimer le contenu existant déjà dans le fichier si nécessaire 
mutation = ['del_ins', 'deletion', 'duplication', 'insertion', 'substitution', 'chevauchement', 'rejected', 'autre', 'unknow', 'inversion', 'incomplete'] # Cette liste contient tous les types de mutations
for element in mutation: # Itération sur chaque élément présent dans la liste
    nom_fichier = f"../result/mutation_{element}.tsv"
    with open(nom_fichier, mode='w') as file:
        file.write('') # Supprime le contenu du fichier


ligne_ajoutee = set() # Contrôle que chaque mutation est traité une seule fois, malgré toutes les loops
data = '../data/Cosmic_MutantCensus_v101_GRCh38.tsv' # Catalogue COSMICS: toutes les mutations de tous les gènes
with open(data, 'r') as file: 
    for line in file: 
        colonne = line.split('\t') 
        for gene_name in gene_dic2: # Itération sur les noms de gène COSMICS
            if colonne[0] == gene_name: # Conserve les lignes sur le gène présentement itéré
                mutation_type = colonne[11] # missense, nonsense, inframe, frameshift, etc.
                transcrit_reference = les_transcrits_de_reference[gene_name]
                mutation_grand = colonne[16] # Grande position CHROMOSOMIQUE de la mutation
                mutation_petit = colonne[15] # Petite position CHROMOSOMIQUE de la mutation
                mutation_explain = colonne[9] # Description de la mutation 
                mutation_id = colonne[8] # Identifiant de la mutation (numéros)
                mutation_descrip = colonne[22] # Autre descriptif de la mutation, utile pour les duplications
                
                # Gérer les mutations Unknow: elles sont retirés puisqu'elles ne possèdent pas assez d'information pour être exécutée
                if mutation_grand == '' or mutation_petit == '':
                    nom_fichier = "../result/mutation_unknow.tsv"
                    with open(nom_fichier, mode='a') as unknow:
                        unknow.write(line)
                        
                else:
                    
                    # Séparation des mutations qui tombent dans un transcrit, dans les introns ou qui chevauchent les exons et les introns (souvent des splice_donor ou splice_acceptor)
                    for exon in gene_dic2[gene_name][transcrit_reference]: # Itération sur chaque exon faisant partie du transcrit de référence

                        # La mutation doit tomber dans un exon du transcrit de référence! N.B: J'enlève les mutations splice_acceptor, splice_donnor...
                        if ((int(mutation_grand) >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and int(mutation_grand) <= int(gene_dic2[gene_name][transcrit_reference][exon][1])) and (int(mutation_petit) >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and int(mutation_petit) <= int(gene_dic2[gene_name][transcrit_reference][exon][1]))) and not mutation_id in ligne_ajoutee and 'splice' not in mutation_type:

                            # Séparation des types de mutations. Les types de mutations sont triés dans des fichiers .tsv commun pour tous les gènes
                            # Si la mutation est une substitution:
                            if '>' in mutation_explain:
                                nom_fichier = "../result/mutation_substitution.tsv"
                                with open(nom_fichier, mode='a') as f:
                                    f.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage
                                
                                    
                            # Si la mutation est une délétion:    
                            elif mutation_explain.endswith('del'):
                                nom_fichier = f"../result/mutation_deletion.tsv"
                                with open(nom_fichier, mode='a') as fichier:
                                    fichier.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage
                                
                            
                            # Si la mutation est une duplication:
                            elif mutation_explain.endswith('dup'):

                                # Duplication de plusieurs nucléotides
                                if '_' in mutation_descrip:
                                    match = re.search(r'g\.(\d+)_([\d]+)', mutation_explain) # Recherche la position chromosomique de départ et de fin des nucléotides dupliqués
                                    if match:
                                        small_pos = int(match.group(1)) # Petite position chromosomique 
                                        big_pos = int(match.group(2)) # Grande position chromosomique
                                        for exon in gene_dic2[gene_name][transcrit_reference]:
                                            if (small_pos >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and small_pos <= int(gene_dic2[gene_name][transcrit_reference][exon][1])) and (big_pos >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and big_pos <= int(gene_dic2[gene_name][transcrit_reference][exon][1])) and not mutation_id in ligne_ajoutee:
                                                nom_fichier = "../result/mutation_duplication.tsv"
                                                with open(nom_fichier, mode='a') as document:
                                                    document.write(line)
                                                    ligne_ajoutee.add(mutation_id) # Marquage
                                            else:
                                                if not mutation_id in ligne_ajoutee:
                                                    nom_fichier = f"../result/mutation_chevauchement.tsv"
                                                    with open(nom_fichier, mode='a') as document:
                                                        document.write(line)
                                                        ligne_ajoutee.add(mutation_id) # Marquage
                            # Duplication d'un nucléotide
                            elif not '_' in mutation_descrip:
                                nom_fichier = f"../result/mutation_duplication.tsv"
                                with open(nom_fichier, mode='a') as document:
                                    document.write(line)
                                    ligne_ajoutee.add(mutation_id)
                                        
                            # Si la mutation est une insertion:
                            elif 'ins' in mutation_explain and not 'delins' in mutation_explain:
                                nom_fichier = "../result/mutation_insertion.tsv"
                                with open(nom_fichier, mode='a') as papier:
                                    papier.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage

                            # Si la mutation est une délétion-insertion:
                            elif 'delins' in mutation_explain:
                                nom_fichier = "../result/mutation_del_ins.tsv"
                                with open(nom_fichier, mode='a') as tableau:
                                    tableau.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage

                            # Si la mutation est une inversion:
                            elif mutation_explain.endswith('inv'):
                                nom_fichier = "../result/mutation_inversion.tsv"
                                with open(nom_fichier, mode='a') as fich:
                                    fich.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage
                    
                            # Si la mutation n'entre pas dans une des catégories précédentes. En principe, si toutes les catégories ont été définies, ce fichier devrait être vide ou inexistant
                            else:
                                if not mutation_id in ligne_ajoutee:
                                    nom_fichier = "../result/mutation_autre.tsv"
                                    with open(nom_fichier, mode='a') as paper:
                                        paper.write(line)
                                        ligne_ajoutee.add(mutation_id) # Marquage
                        
                        else:
                            if ((int(mutation_grand) >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and int(mutation_grand) <= int(gene_dic2[gene_name][transcrit_reference][exon][1])) or (int(mutation_petit) >= int(gene_dic2[gene_name][transcrit_reference][exon][0]) and int(mutation_petit) <= int(gene_dic2[gene_name][transcrit_reference][exon][1]))) and not mutation_id in ligne_ajoutee:
                                nom_fichier = "../result/mutation_chevauchement.tsv"
                                with open(nom_fichier, mode='a') as f:
                                    f.write(line)
                                    ligne_ajoutee.add(mutation_id) # Marquage 
                                
                    else:
                        if not mutation_id in ligne_ajoutee: # Retient les lignes qui n'ont pas été ajouté seulement
                            nom_fichier = "../result/mutation_rejected.tsv"
                            with open(nom_fichier, mode='a') as fichier:
                                fichier.write(line)
                                ligne_ajoutee.add(mutation_id) 
                                
                                
                if mutation_id not in ligne_ajoutee:
                    nom_fichier = f'../result/mutation_incomplete.tsv'
                    with open(nom_fichier, mode='a') as fichier:
                        fichier.write(line)
                        ligne_ajoutee.add(mutation_id)
                        
                        
# Description du contenu des fichiers obtenus
                      
# substitution: mutation de type substitution
# del_ins: mutation de type del_ins
# deletion: mutation de type deletion
# insertion: mutation de type insertion
# inversion: mutation de type inversion
# duplication: mutation de type duplication
# autre: autres de cas de mutation non-traité, doit être vide
# chevauchement: mutation en chevauchement intron-exon (très souvent splice_donnor, splice_acceptor)
# unknow et incomple: pas assez de détail sur la mutation pour être effectué (ex. pas de position)
# rejected: mutation dans les introns


In [6]:
# RÉPARTITION DES MUTATIONS POUVANT ÊTRE APPLIQUÉ POUR CHAQUE TRANSCRIT DES GÈNES

count_les_transcrit_mute_par_gene = {}
traited = ['duplication', 'substitution', 'insertion', 'deletion', 'inversion', 'del_ins']                 
for gene_name in gene_dic2: 
    gene = equivalence_nom[gene_name] # Sort l'équivalent ENSG
    count_les_transcrit_mute_par_gene[gene_name] = {} # Ajoute le nom usuel COSMICS comme clé
    for transcrit in gene_dic2[gene_name]:
        count_les_transcrit_mute_par_gene[gene_name][transcrit] = [] # Ajoute les transcrits sous chaque gène
    
        for item in traited:
            with open(f'../result/mutation_{item}.tsv', 'r') as file : # Ouverture des fichiers contenant les mutations triés     
                for line in file:
                    colonne = line.split('\t')
                    if colonne[0] == gene_name: # Conserve les lignes sur le gène présentement itéré
                        mutation_petite = int(colonne[15]) # Petite position chromosomique de la mutation
                        mutation_grande = int(colonne[16]) # Grande position chromosomique de la mutation
                        mutation_id = colonne[8] # Identifiant de la mutation (numéro)
                        
                    
                        for exon in gene_dic2[gene_name][transcrit]:
                            # La mutation doit se retrouver dans un exon du transcrit    
                            if (mutation_petite >= int(gene_dic2[gene_name][transcrit][exon][0]) and mutation_petite <= int(gene_dic2[gene_name][transcrit][exon][1])) and (mutation_grande >= int(gene_dic2[gene_name][transcrit][exon][0]) and mutation_grande <= int(gene_dic2[gene_name][transcrit][exon][1])):
                                count_les_transcrit_mute_par_gene[gene_name][transcrit].append(line) # Ajoute la ligne de la mutation    
#print(count_les_transcrit_mute_par_gene)

In [46]:
##### PRODUIRE CHAQUE TRANSCRIT AVEC CHAQUE MUTATION APPLICABLE #####
# Cette section produit toutes les séquences d'ADNc des transcrits avec chaque mutation de n'importe quelle catégorie qui sont applicables.
# Les séquences mutés sont ajoutés dans des fichiers fasta triés en fonction du type de mutation effectué 


# Cette partie sert à effacer le contenu des fichiers fasta déjà existant
for item in traited:
    nom_fichier = f"../result/{item}.fasta"
    with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
        file.write('') # Supprime le contenu du fichier
nom_fichier = "../result/mutation_not_working.tsv"
with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
    file.write('') # Supprime le contenu du fichier
nom_fichier = "../result/transcrit_no_mutation.fasta"
with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
    file.write('') # Supprime le contenu du fichier
nom_fichier = '../result/all_transcripts_with_mutation.fasta'
with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
    file.write('') # Supprime le contenu du fichier
    
for gene_name in gene_dic2:
    gene = equivalence_nom[gene_name] # Sort l'équivalent en ENSG
    for transcrit in gene_dic2[gene_name]:
        sequence = sequence_cDNA(gene, transcrit) # Sort la séquence d'ADNc
        header1 = '>' + f'{gene}_{transcrit}' # Formation du header
        fasta = f'../result/transcrit_no_mutation.fasta'
        with open(fasta, 'a') as f:
            f.write(header1 + '\n') # Écriture du header
            f.write(sequence + '\n') # Écriture de la séquence

lignes_deja_ajoutees = set() # Conserve les lignes de mutations défectueuses
for gene_name in gene_dic2:
    gene = equivalence_nom[gene_name] # Sort l'équivalent en ENSG
    for transcrit in gene_dic2[gene_name]:
        # TRANSCRITS SANS MUTATION
        #if f'{gene}_{transcrit}' not in sans_mutation_fait:
        sequence = sequence_cDNA(gene, transcrit) # Sort la séquence d'ADNc
            #header1 = '>' + f'{gene}_{transcrit}' # Formation du header
            #fasta = f'../result/transcrit_no_mutation.fasta'
            #with open(fasta, 'a') as f:
                #f.write(header1 + '\n') # Écriture du header
                #f.write(sequence + '\n') # Écriture de la séquence
                #sans_mutation_fait.add(line)
        for line in count_les_transcrit_mute_par_gene[gene_name][transcrit]: # Itére sur chaque ligne de mutation applicable pour le transcrit
            colonne = line.split('\t')
            changement = colonne[9] # Description de la mutation
            position_petite = int(colonne[15]) # Petite position chromosomique de la mutation
            position_grande = int(colonne[16]) # Grande position chromosomique de la mutation
            mutation_explain = colonne[22] # Utile pour les duplications
            mutation_id = colonne[8] # Identifiant de la mutation

            # Conversion des positions chromosomiques de la mutation en position chronologique dans le transcrit COMMENÇANT PAR 0
            position_chrono_1 = (get_position_chronologique(gene, position_petite, transcrit))-1 
            position_chrono_2 = (get_position_chronologique(gene, position_grande, transcrit))-1

            liste = [position_chrono_1, position_chrono_2] # Ajoute les deux positions chronologiques dans la liste
            brin = colonne[17] # Nature du brin +/-

            if brin == '+':
                delete = colonne[23] # Stock la séquence nucléotidique qui est retirée
                add = colonne[24] # Stock la séquence nucléotidique qui est ajoutée

            if brin == '-': # Si le gène est sur le brin -, la database donne la séquence du brin inverse et complémentaire (brin +)
                enleve = colonne[23]
                chaine_inverse = enleve[::-1] # Inversion (miroir) de la chaine
                # Obtention de la séquence complémentaire 
                adn1 = chaine_inverse.replace('A', 't') 
                adn2 = adn1.replace('T', 'a')
                adn3 = adn2.replace('C', 'g')
                adn4 = adn3.replace('G', 'c')
                delete = adn4.upper()

                ajoute = colonne[24]
                chaine_miroir = ajoute[::-1] # Inversion (miroir) de la chaine
                # Obtention de la séquence complémentaire
                adn5 = chaine_miroir.replace('A', 't')
                adn6 = adn5.replace('T', 'a')
                adn7 = adn6.replace('G', 'c')
                adn8 = adn7.replace('C', 'g')
                add = adn8.upper()
                
            # GÉRER LES SUBSTITUTIONS    
            if position_petite == position_grande and '>' in changement:
                position_chrono = (get_position_chronologique(gene, position_petite, transcrit))-1
                base_avant = changement[-3] # Stockage de la base initialement contenu dans le transcrit
                base_apres = changement[-1] # Stockage de la base qui va venir remplacée l'ancienne base
                if sequence[position_chrono]== base_avant: # Vérification
                    sequence_modifiee = sequence[:position_chrono]+ base_apres + sequence[position_chrono+1:] # Formation de la séquence mutée
                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                    fasta = f'../result/substitution.fasta'
                    with open(fasta, 'a') as f:
                        f.write(header + '\n') # Écriture du header
                        f.write(sequence_modifiee + '\n') # Écriture de la séquence
                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                else: 
                    if line not in lignes_deja_ajoutees:
                        reste = f'../result/mutation_not_working.tsv'
                        with open(data, 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses en mode append()
                            r.write(line) # Écriture de la ligne dans le fichier
                            lignes_deja_ajoutees.add(line)

            
            # GÉRER LES DELETIONS
            elif changement.endswith('del'):
                smallest = min(liste) # Petite position chronologique = DÉBUT DE LA MUTATION
                biggest = max(liste) # Grande position chronologique = FIN DE LA MUTATION
                remove_sequence = sequence[smallest:biggest+1] 
                if remove_sequence == delete: # Vérification
                    sequence_modifiee = sequence[:smallest] + sequence[biggest+1:] # Formation de la séquence modifiée
                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                    fasta = f'../result/deletion.fasta' 
                    with open (fasta, 'a') as f:
                        f.write(header + '\n') # Écriture du header
                        f.write(sequence_modifiee + '\n') # Écriture de la séquence
                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                else: 
                    if line not in lignes_deja_ajoutees:
                        reste = f'../result/mutation_not_working.tsv'
                        with open(reste, 'a') as r: # Ouverture du fichier qui récupère les mutations défectueuses en mode append()
                            r.write(line) # Écriture de la ligne dans le fichier
                            lignes_deja_ajoutees.add(line) 

            # GÉRER LES INSERTIONS
            elif 'ins' in changement and not 'delins' in changement: 
                smallest = min(liste) # Petite position chronologique = position de départ de la mutation
                biggest = max(liste) # Grande position chronologique = position de fin de la mutation
                pattern = r'[ATCG]+' 
                match = re.search(pattern, changement)
                sequence_ajoutee = match.group()
                if sequence_ajoutee == add: # Vérification
                    sequence_modifiee = sequence[:biggest] + sequence_ajoutee + sequence[biggest:] # Formation de la séquence mutés
                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                    fasta = f'../result/insertion.fasta'
                    with open (fasta, 'a') as f: 
                        f.write(header + '\n') # Écriture du header
                        f.write(sequence_modifiee + '\n') # Écriture de la séquence
                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                else: 
                    if line not in lignes_deja_ajoutees: 
                        reste = f'../result/mutation_not_working.tsv'
                        with open(reste, 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses
                            r.write(line) # Écriture de la ligne de la mutation
                            lignes_deja_ajoutees.add(line)

            # GÉRER LES DUPLICATIONS
            elif changement.endswith('dup'):
                    
            # Le script est différent si on a une duplication d'un nucléotide ou de plusieurs nucléotides
                # PLUSIEURS NUCLÉOTIDES DUPLIQUÉS
                if '_' in mutation_explain:
                    match = re.search(r'g\.(\d+)_([\d]+)', mutation_explain) # Recherche la position chromosomique de départ et de fin des nucléotides dupliqués
                    if match:
                        small_pos = int(match.group(1)) # Petite position chromosomique 
                        big_pos = int(match.group(2)) # Grande position chromosomique
                        # Conversion des positions chromosomiques en position chronologique
                        chrono_1 = (get_position_chronologique(gene, small_pos, transcrit)-1)
                        chrono_2 = (get_position_chronologique(gene, big_pos, transcrit)-1)
                        liste_position_chrono = [chrono_1, chrono_2]
                        biggest = max(liste_position_chrono) # Grande position chronologique: fin de la mutation
                        smallest = min(liste_position_chrono) # Petite position chronologique: début de la mutation
                        sequence_dupliquee = sequence[smallest: biggest+1]
                        if sequence_dupliquee == add: # Vérification
                            sequence_modifiee = sequence[:biggest+1] + sequence_dupliquee + sequence[biggest+1:] # Formation de la séquence mutées
                            header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                            fasta = f'../result/duplication.fasta'
                            with open (fasta, mode = 'a') as f:
                                f.write(header + '\n') # Écriture du header
                                f.write(sequence_modifiee + '\n') # Écriture de la séquence
                        # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                        else: 
                            if line not in lignes_deja_ajoutees:
                                reste = f'../result/mutation_not_working.tsv'
                                with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses en mode append()
                                    r.write(line) # Écriture de la ligne de la mutation
                                    lignes_deja_ajoutees.add(line)
                else:
                    # 1 NUCLÉOTIDE DUPPLIQUÉ
                    if not '_' in mutation_explain:
                        match = re.search(r'g\.(\d+)dup', mutation_explain) # Recherche la position chromosomique du nucléotide qui sera dupliqué
                        if match:
                            position = int(match.group(1)) # Position chromosomique du nucléotide dupliqué
                            chrono_position = int(get_position_chronologique(gene, position, transcrit)-1)
                            sequence_dupliquee = sequence[chrono_position]
                            if sequence_dupliquee == add: # Vérification
                                sequence_modifiee = sequence[:chrono_position+1] + sequence_dupliquee + sequence[chrono_position+1:] # Formation de la séquence mutée
                                header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                                fasta = f'../result/duplication.fasta' 
                                with open (fasta, mode = 'a') as f:
                                    f.write(header + '\n') # Écriture du header
                                    f.write(sequence_modifiee + '\n') # Écriture de la séquence
                            # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                            else:
                                if line not in lignes_deja_ajoutees: 
                                    reste = f'../result/mutation_not_working.tsv'
                                    with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses
                                        r.write(line) # Écriture de la ligne de la mutation
                                        lignes_deja_ajoutees.add(line)

            # GÉRER LES DELETION-INSERTION
            elif 'delins' in changement:
                # Traitement de la séquence deletée
                smallest = min(liste) # Petite position chronologique = début de la mutation
                biggest = max(liste) # Grande position chronologique = fin de la mutation
                remove_sequence = sequence[smallest:biggest+1]

                # Traitement de la séquence ajoutée
                pattern = r'[ATCG]+'
                match = re.search(pattern, changement) # Recherche la séquence ajoutée
                sequence_ajoutee = match.group() 

                if remove_sequence == delete and sequence_ajoutee == add: # Vérification
                    sequence_modifiee = sequence[:smallest] + sequence_ajoutee + sequence[biggest+1:] # Formation de la séquence mutée
                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                    fasta = f'../result/del_ins.fasta'
                    with open (fasta, 'a') as f: 
                        f.write(header + '\n') # Écriture du header
                        f.write(sequence_modifiee + '\n') # Écriture de la séquence
                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                else: 
                    if line not in lignes_deja_ajoutees:
                        reste = f'../result/mutation_not_working.tsv'
                        with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses en mode append()
                            r.write(line) # Écriture de la ligne de la mutation
                            lignes_deja_ajoutees.add(line)

            # GÉRER LES INVERSIONS
            elif changement.endswith('inv'):
                smallest = min(liste) # Petite position chronologique = début de la mutation
                biggest = max(liste) # Grande position chronologique = fin de la mutation
                sequence_inverse = sequence[smallest: biggest+1] 
                if sequence_inverse == delete: # Vérification
                    sequence_modifiee = sequence[:smallest] + sequence_inverse[::-1] + sequence[biggest+1:] # Formation de la séquence mutée
                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                    fasta = f'../result/inversion.fasta' 
                    with open (fasta, 'a') as f:
                        f.write(header + '\n') # Écriture du header
                        f.write(sequence_modifiee + '\n') # Écriture du header
                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                else:
                    if line not in lignes_deja_ajoutees:
                        reste = f'../result/mutation_not_working.tsv'
                        with open(reste, 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses en mode append()
                            r.write(line) # Écriture de la ligne de la mutation
                            lignes_deja_ajoutees.add(line)


In [47]:
##### ÉTAPE 8: AJOUTER TOUS LES TRANSCRITS MUTÉS DANS UN GROS FASTA #####
# Cette section permet de combiner tous les sous-fichier fasta qui contiennent les séquences mutés et non-muté classées selon chaque type de mutation, dans
# un seul gros fichier fasta.
    
# Liste de tous les fichiers fasta voulant être combiné
fichiers_fasta = ['../result/substitution.fasta', '../result/insertion.fasta', '../result/deletion.fasta', '../result/duplication.fasta', '../result/del_ins.fasta', '../result/inversion.fasta', '../result/transcrit_no_mutation.fasta']
# Le nom du fichier qui va contenir tous les contenus
fichier_sortie = '../result/all_transcripts_with_mutation.fasta'

with open(fichier_sortie, 'w') as fichier_combinaison: # Ouverture du fichier qui va contenir tous les contenus
    for fichier in fichiers_fasta:
        try:
            with open(fichier, 'r') as f: 
                fichier_combinaison.write(f.read()) # Écrit le contenu du fichier dans le gros fichier
        except FileNotFoundError: # Cela se peut, si exemple, le gène ne contient pas de mutation inversion
            continue

In [48]:
# ÉTAPE DE VÉRIFICATION
# Le nombre de transcrit obtenu POUR UN GÈNE est égal au nombre de toutes les mutations qui s'effectue pour chaque transcrit + tous les transcrits
# à l'état wild-type - le nombre de mutation erroné qui ne s'effectuent pas (doit être compté pour chaque transcrit pour lequel la mutation 
# s'appliquait)


for gene_name in gene_dic2:
    count = 0 # Compte le nombre de transcrit pour un gène
    count_defectueux = 0 # Compte le nombre de fois qu'une mutation erroné ne s'est pas effectuée
    gene = equivalence_nom[gene_name]
    
    # Compte combien de transcrits ont été produit pour chaque gène
    with open('../result/all_transcripts_with_mutation.fasta', 'r') as file:
        for line in file:
            if gene in line:
                count += 1
    
    # Compte combien de fois une mutation erronée est retrouvé dans un gène
    for line in lignes_deja_ajoutees: # Le set contient les lignes erronées
        colonne = line.split('\t')
        mutation_id = colonne[8]
        if colonne[0] == gene_name:
            for transcrit in gene_dic2[gene_name]:
                for item in count_les_transcrit_mute_par_gene[gene_name][transcrit]:
                    col = item.split('\t')
                    if col[8]== mutation_id:
                        count_defectueux += 1
                        print(count_defectueux)
    
    # Compte en théorie combien on devrait avoir de transcrit muté        
    nombre_de_mutation = sum(len(liste) for liste in count_les_transcrit_mute_par_gene[gene_name].values())
    total = nombre_de_mutation + len(gene_dic2[gene_name])
    
    if (nombre_de_mutation + len(gene_dic2[gene_name]))== count-count_defectueux:
        print(f'Correct pour {gene_name} ' + str(total) + '==' + str(count-count_defectueux))
    else:
        print(f'Error pour {gene_name} ' + str(total) + '!=' + str(count-count_defectueux))


Correct pour PRDM16 5133==5133
Correct pour THRAP3 1463==1463
Correct pour CDKN2C 482==482
Correct pour TENT5C 490==490
Correct pour TPR 2042==2042
Correct pour PRRX1 1090==1090
Correct pour ARHGEF10L 2698==2698
